In [4]:
import os
import json
import torch

args_perm_id = 0
args_task_num = 5
args_class_num = 10
args_shot_num = 5
def collect_from_json(dataset, root, split):
    if split == "train":
        pth = os.path.join(
            root,
            dataset,
            "perm" + str(args_perm_id),
            f"{dataset}_{args_task_num}task_{args_class_num // args_task_num}way_{args_shot_num}shot.{split}.jsonl",
        )
    elif split in ["dev", "test"]:
        pth = os.path.join(root, dataset, f"{dataset}.{split}.jsonl")
    elif split == "stream":
        pth = os.path.join(
            root,
            dataset,
            f"stream_label_{args_task_num}task_{args_class_num // args_task_num}way.json",
        )
    else:
        raise ValueError(f'Split "{split}" value wrong!')
    if not os.path.exists(pth):
        raise FileNotFoundError(f"Path {pth} do not exist!")
    else:
        with open(pth) as f:
            if pth.endswith(".jsonl"):
                data = [json.loads(line) for line in f]
                if split == "train":
                    data = [list(i.values()) for i in data]
            else:
                data = json.load(f)
    # if split == "train":
    #     data = extract_single_dict(data)

    return data

In [8]:
dataset = 'ACE'
root = './data_incremental'
split = 'train'

In [24]:
data = [instance for t in collect_from_json(dataset, root, split)[1] for instance in t]

In [64]:
import torch


def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    trigger_word = []
    seq_len = len(data_instance["piece_ids"]) + 1 # because start_index of piece_ids is 1 instead of 0
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            trigger_word.append(data_instance["span"][i])

    set_label_in_one_sentence = set(true_label)
    true_one_hot_trigger_vector = torch.zeros(num_label)
    for i in set_label_in_one_sentence:
        true_one_hot_trigger_vector += torch.eye(num_label)[i]

    true_one_hot_label_vector = torch.zeros(seq_len)
    trigger = []
    for i in trigger_word:
        trigger.extend(i)

    set_trig = set(trigger)
    for i in set_trig:
        true_one_hot_label_vector += torch.eye(seq_len)[i]
    return true_one_hot_trigger_vector, true_one_hot_label_vector

In [65]:
a, b = get_one_hot_true_label_and_true_trigger(data[0],10)

In [67]:
a

tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0.])

In [68]:
b

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.])

In [52]:
print(len(data[0]['piece_ids']))
print(len(data[0]['label']))
print(len(data[0]['span']))

50
47
47


In [72]:
for instance in data:
    true_one_hot_trigger_vector, true_one_hot_label_vector = get_one_hot_true_label_and_true_trigger(instance, 10)
    instance['true_one_hot_trigger_vector'] = true_one_hot_trigger_vector.tolist()
    instance['true_one_hot_label_vector'] = true_one_hot_label_vector.tolist()

In [74]:
data[0]

{'piece_ids': [101,
  3960,
  1010,
  1045,
  2228,
  2008,
  1996,
  3114,
  7955,
  1999,
  1996,
  2148,
  1011,
  1011,
  2017,
  2113,
  1010,
  2034,
  1997,
  2035,
  1010,
  2057,
  2020,
  1011,
  1011,
  2043,
  5951,
  8573,
  2001,
  2700,
  2343,
  1010,
  2057,
  2018,
  2042,
  2542,
  2054,
  2057,
  2245,
  2001,
  2145,
  1037,
  11438,
  3842,
  2044,
  1996,
  2942,
  2162,
  1012,
  102],
 'label': [6,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'span': [[29, 29],
  [46, 47],
  [1, 1],
  [2, 2],
  [3, 3],
  [4, 4],
  [5, 5],
  [6, 6],
  [7, 7],
  [8, 8],
  [9, 9],
  [10, 10],
  [11, 11],
  [12, 12],
  [13, 13],
  [14, 14],
  [15, 15],
  [16, 16],
  [17, 17],
  [18, 18],
  [19, 19],
  [20, 20],
  [21, 21],
  [22, 22],
  [23, 23],
  [24, 24],
  [25, 25],
  [26, 26],
  [27, 27],
  [2

In [75]:
torch.cuda.is_available()

False

In [ ]:
import torch


def get_one_hot_true_label_and_true_trigger(data_instance, num_label):
    true_label = []
    trigger_word = []
    seq_len = len(data_instance["piece_ids"]) + 1 # because start_index of piece_ids is 1 instead of 0
    for i in range(len(data_instance["label"])):
        if data_instance["label"][i] != 0:
            true_label.append(data_instance["label"][i])
            trigger_word.append(data_instance["span"][i])

    set_label_in_one_sentence = set(true_label)
    true_one_hot_trigger_vector = torch.zeros(num_label)
    for i in set_label_in_one_sentence:
        true_one_hot_trigger_vector += torch.eye(num_label)[i]

    true_one_hot_label_vector = torch.zeros(seq_len)
    trigger = []
    for i in trigger_word:
        trigger.extend(i)

    set_trig = set(trigger)
    for i in set_trig:
        true_one_hot_label_vector += torch.eye(seq_len)[i]
    return true_one_hot_trigger_vector, true_one_hot_label_vector

In [ ]:
import torch
def true_label_and_trigger(train_x,train_y,train_masks, train_span, class_num):
    num_instance = len(train_x)
    true_one_hot_label_vectors = []
    true_one_hot_trigger_vectors = []
    for i in range(num_instance):
        data_instace={
            'piece_ids': train_x[i],
            'label': train_y[i],
            'span': train_span[i],
            'mask': train_masks[i]
        }

        true_one_hot_trigger_vector, true_one_hot_label_vector = get_one_hot_true_label_and_true_trigger(data_instance=data_instace,num_label=class_num)
        true_one_hot_trigger_vectors.append(true_one_hot_trigger_vector)
        true_one_hot_label_vectors.append(true_one_hot_label_vector)

        